In [1]:
# python库导入
import numpy as np
import pandas as pd

In [2]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import model_selection
from sklearn import metrics
from sklearn.grid_search import GridSearchCV

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

In [5]:
train = pd.read_csv('data/train_modified.csv')

In [6]:
print(train.shape)

(87020, 51)


In [7]:
print(train.head())

   Disbursed  Existing_EMI           ID  Loan_Amount_Applied  \
0        0.0           0.0  ID000002C20             300000.0   
1        0.0           0.0  ID000004E40             200000.0   
2        0.0           0.0  ID000007H20             600000.0   
3        0.0           0.0  ID000008I30            1000000.0   
4        0.0       25000.0  ID000009J40             500000.0   

   Loan_Tenure_Applied  Monthly_Income  Var4  Var5  Age  \
0                  5.0           20000     1     0   37   
1                  2.0           35000     3    13   30   
2                  4.0           22500     1     0   34   
3                  5.0           35000     3    10   28   
4                  2.0          100000     3    17   31   

   EMI_Loan_Submitted_Missing    ...     Var2_2  Var2_3  Var2_4  Var2_5  \
0                           1    ...        0.0     0.0     0.0     0.0   
1                           0    ...        0.0     0.0     0.0     0.0   
2                           1    ..

In [8]:
target = 'Disbursed'
IDcol = 'ID'

In [11]:
# 自定义函数：创建模型和交叉验证
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['Disbursed'],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print("\nModel Report")
    print("Accuracy : %.4g" % metrics.accuracy_score(dtrain['Disbursed'].values, dtrain_predictions))
    print("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['Disbursed'], dtrain_predprob))
                    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [12]:
#Choose all predictors except target & IDcols
predictors = [x for x in train.columns if x not in [target, IDcol]]
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb1, train, predictors)


Model Report
Accuracy : 0.9854
AUC Score (Train): 0.895159


TypeError: 'str' object is not callable

In [15]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier
from xgboost.sklearn import XGBClassifier
import lightgbm